In [1]:
# Laura Chapman
# Script for developing statistical tools for Glue
# Computes statistics for subsets as well as entire data using compute_statistic
# Changed from astropy tables to pandas dataframe

# Format data well in a popup using qt
# Color code by subset

In [1]:
# Basic code that imports glue and loads in and links the data

import sys
from glue.core.data_factories import load_data
from glue.core import DataCollection
from glue.core.link_helpers import LinkSame
from glue.app.qt.application import GlueApplication
from glue.viewers.image.qt import ImageViewer
from glue_vispy_viewers.volume.volume_viewer import VispyVolumeViewer

image_filename='w5.fits'
catalog_filename='w5_psc.vot'

#load 2 datasets from files
catalog = load_data(catalog_filename)
image = load_data(image_filename)

dc = DataCollection([catalog,image])

# link positional information
dc.add_link(LinkSame(catalog.id['RAJ2000'], image.id['Right Ascension']))
dc.add_link(LinkSame(catalog.id['DEJ2000'], image.id['Declination']))

#Create subset based on filament mask
ra_state=(image.id['Right Ascension'] > 44) & (image.id['Right Ascension'] < 46)
subset_group=dc.new_subset_group('RA_Selection',ra_state)
subset_group.style.color = '#0000FF'

#start Glue
# app = GlueApplication(dc)

# imageviewer = app.new_data_viewer(ImageViewer)
# imageviewer.add_data(image)

# app.start()



In [2]:
import glue.utils.array as gua
import glue.core.data as gcd
from astropy.table import Table

In [3]:
# Constructs a pandas DataFrame instead of an astropy table

import pandas as pd
from pandas import DataFrame
import numpy as np

In [4]:
# Only calculates statistics for one of the data sets
# Displays using a pandas DataFrame

data = dc[0]

mean_array = []
median_array = []
min_array = []
max_array = []
sum_array = []
name_array = []
tables = []
headings = ('mean', 'median', 'minimum', 'maximum', 'sum')

print(data.label)

for j in range (0, len(data.components)):
    name = data.components[j].label # Get the name of each component
    name_array.append(name) # add to the name array to build the table
    mean_array.append(data.compute_statistic('mean', data.components[j]))
    median_array.append(data.compute_statistic('median', data.components[j]))       
    min_array.append(data.compute_statistic('minimum', data.components[j]))       
    max_array.append(data.compute_statistic('maximum', data.components[j]))      
    sum_array.append(data.compute_statistic('sum', data.components[j]))         

    # At this point we have several column arrays to put into a pandas data frame
    
column_data = np.asarray([mean_array, median_array, min_array, max_array, sum_array]).transpose()

pd.DataFrame(column_data, index=name_array, columns=headings)

w5_psc


mean       median    minimum       maximum  \
Pixel Axis 0 [x]  8885.000000  8885.000000   0.000000  17770.000000   
World 0           8885.000000  8885.000000   0.000000  17770.000000   
ID                8886.000000  8886.000000   1.000000  17771.000000   
RAJ2000             43.531802    43.468829  41.081526     46.039649   
DEJ2000             60.340595    60.382988  59.246093     61.493241   
Jmag                13.563952    13.760000   5.120000     17.459999   
Hmag                12.923938    13.170000   3.890000     16.450001   
Ksmag               12.667512    12.940000   3.510000     16.049999   
__3.6_              12.511145    12.800000   4.860000     16.620001   
__4.5_              12.449672    12.740000   3.980000     16.129999   
__5.8_              12.355568    12.650000   2.140000     15.020000   
__8.0_              12.229861    12.550000   2.560000     14.730000   
__24_                8.014891     8.330000   0.280000     11.500000   
Type                 2.882899     3.000000   0.000000      4.000000   
__4.5__-__5.8_       0.093754     0.060000  -2.140000      2.800000   
__5.8__-__8.0_       0.124974     0.040000  -0.890000      1.960000   

                           sum  
Pixel Axis 0 [x]  1.578953e+08  
World 0           1.578953e+08  
ID                1.579131e+08  
RAJ2000           7.736036e+05  
DEJ2000           1.072313e+06  
Jmag              2.317401e+05  
Hmag              2.222142e+05  
Ksmag             2.167665e+05  
__3.6_            2.223355e+05  
__4.5_            2.212431e+05  
__5.8_            2.189777e+05  
__8.0_            2.158815e+05  
__24_             1.271963e+04  
Type              5.123200e+04  
__4.5__-__5.8_    1.661597e+03  
__5.8__-__8.0_    2.205165e+03

In [5]:
# Defines a subset and runs statistics using compute_statistic for the subset
# Saves data using a pandas dataframe called my_pandas_data

# Data that subset is pulled from
data = dc[0]

# Define state and subset
state1 = data.id['Jmag'] > 14
subset1 = data.new_subset(state1, label='Jmag > 14')

# Same arrays as with full data
mean_array = []
median_array = []
min_array = []
max_array = []
sum_array = []
name_array = []
tables = []

headings = ('mean', 'median', 'minimum', 'maximum', 'sum')

print(data.label, 'subset1')
print() 
for j in range (0, len(data.components)):
    name = data.components[j].label # Get the name of each component
    name_array.append(name) # add to the name array to build the table
    mean_array.append(data.compute_statistic('mean', subset1.components[j], subset_state=subset1.subset_state))
    median_array.append(data.compute_statistic('median', subset1.components[j], subset_state=subset1.subset_state))       
    min_array.append(data.compute_statistic('minimum', subset1.components[j], subset_state=subset1.subset_state))       
    max_array.append(data.compute_statistic('maximum', subset1.components[j], subset_state=subset1.subset_state))      
    sum_array.append(data.compute_statistic('sum', subset1.components[j], subset_state=subset1.subset_state))        
   
column_data = np.asarray([mean_array, median_array, min_array, max_array, sum_array]).transpose()

my_pandas_data = pd.DataFrame(column_data, index=name_array, columns=headings)

w5_psc subset1



In [7]:
# # Use qtpy to make a popup with statistics
# # Unsure how to feed qt the table directly- this code attempts to do it by indexing through the statistical array
# # Probably not the most efficient solution
# # Uses data from the subset (saved in 2D array column_data)

# # Initialize application
# app = QApplication.instance()
# if app is None:
#     app = QApplication(sys.argv)
# else:
#     print('QApplication instance already exists: %s' % str(app))

# # Create layout and add widgets
# layout = QGridLayout()

# for k in range (0, len(headings)):
#     layout.addWidget(QLabel(headings[k]), 0, k+1)
# for i in range (0, len(column_data)):
#     layout.addWidget(QLabel(name_array[i]), i+1, 0)
#     for j in range (0, len(headings)):
#         layout.addWidget(QLabel(str(column_data[i][j])), i+1, j+1)

# # Apply layout to widget
# widget = QWidget()
# widget.setWindowTitle('Statistics')
# widget.setLayout(layout)

# # Show widget
# widget.show()

# # Start event loop
# app.exec_()

In [8]:
# Edit pandas data model to take in a data collection and display in a drop down 
# For now- have a button with the subsets
# change to calculate one component
# button that says attributes for the components- show a 1 by whatever table
# have the user pick a component and then 

In [9]:
# Full working example! 
# Has a dropdown menu with the components- highlights the row of the component selected
# Still need to figure out how to resize window to fit the contents
# Ideally would merge into one class and take in a data object as a parameter

import sys
from PyQt5 import QtCore, QtWidgets
Qt = QtCore.Qt
from PyQt5.QtCore import QVariant
from qtpy.QtWidgets import (QApplication, QLabel, QWidget, QComboBox,
                            QVBoxLayout, QPushButton, QGridLayout, QTableView)

class PandasModel(QtCore.QAbstractTableModel):
    def __init__(self, df, parent=None):
        QtCore.QAbstractTableModel.__init__(self, parent)
        self.data_frame = df
        super(PandasModel, self).__init__(parent)      

    def rowCount(self, parent=None):
        return len(self.data_frame.values)

    def columnCount(self, parent=None):
        return self.data_frame.columns.size

    def data(self, index, role=Qt.DisplayRole):
        if index.isValid():
            if role == Qt.DisplayRole:
                return QVariant(str(
                    self.data_frame.values[index.row()][index.column()]))
        return QVariant()
    
    def headerData(self, col, orientation, role):
        if orientation == Qt.Horizontal and role == Qt.DisplayRole:
            return QVariant(self.data_frame.columns[col])
        if orientation == Qt.Vertical and role == Qt.DisplayRole:
            return QVariant(self.data_frame.index[col])
        return QVariant()
    
    def selection_pandas(self,i):
        print ("Items in the list are :")

        for count in range(self.cb.count()):
            print (self.cb.itemText(count))
        print ("Current index",i,"selection changed ",self.cb.currentText())    

class combodemo(QWidget):
    def __init__(self, parent = None):
        super(combodemo, self).__init__(parent)
      
        layout = QVBoxLayout()
        self.cb = QComboBox()
    
        for i in range(0, len(my_pandas_data.index)):
            self.cb.addItem(my_pandas_data.index[i])
        
        self.cb.currentIndexChanged.connect(self.selectionchange)
    
        layout.addWidget(self.cb)  
    
        self.view = QTableView()
        model = PandasModel(my_pandas_data)
        self.view.setModel(model)
        self.view.setShowGrid(False)

        layout.addWidget(self.view) 

        self.setLayout(layout)
        self.setWindowTitle("Statistics")    
    
    
        self.setLayout(layout)
        self.setWindowTitle("Statistics")
        self.view.setSelectionBehavior(QTableView.SelectRows)
        
    def selectionchange(self,i):
        print ("Current index",i,"selection changed to",self.cb.currentText())
        self.view.selectRow(i)

app = QApplication.instance()
if app is None:
    app = QApplication(sys.argv)
else:
    print('QApplication instance already exists: %s' % str(app))
    
ex = combodemo()
ex.show()
sys.exit(app.exec_())

Current index 3 selection changed to RAJ2000


SystemExit: 0

/Users/laurachapman/anaconda/envs/glueviz-dev/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
